In [ ]:
## Data preprocessing: Adding number of contacts

In [1]:
import pandas as pd 
import numpy as np

In [2]:
# Load data
data = pd.read_csv("./fravær_merge.csv",sep=";")
data.head()

,LoenafdelingNiveau2Tekst,Aldersgruppe,Stilling_niv1,Fravaeraarsag_gruppe,Koen,Antal_Timer,Antal_Personer,Timestamp,Year,Month,Day,is_weekend,weekday,is_holiday,is_school_holiday,Antal_timer_yesterday,Antal_Loentimer
0,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,26.0,2,2021-01-02,2021,1,2,1,5,0,0,10.0,1173.233835
1,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,0.0,1,2021-01-03,2021,1,3,1,6,0,0,26.0,1173.233835
2,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,10.0,1,2021-01-04,2021,1,4,0,0,0,0,0.0,1173.233835
3,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,10.0,1,2021-01-10,2021,1,10,1,6,0,0,10.0,1173.233835
4,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,10.0,1,2021-01-19,2021,1,19,0,1,0,0,10.0,1173.233835


In [3]:
# Load number of patients in the departments
patient_contact = pd.read_csv("./2024_05_13_Patientkontakter.csv",sep = ";")
patient_contact.head()

,Dato_Ind,Nuv_OrgMapSghTekst,Nuv_OrgMapOverAfdTekst,Antal_Kontakter
0,2022-03-24,REGIONSHOSPITALET GØDSTRUP,MEDICINSK AFDELING - RHG,309
1,2021-07-19,REGIONSHOSPITALET GØDSTRUP,URINVEJSKIRURGI - RHG,57
2,2020-08-06,REGIONSHOSPITALET GØDSTRUP,URINVEJSKIRURGI - RHG,85
3,2019-12-04,REGIONSHOSPITALET GØDSTRUP,URINVEJSKIRURGI - RHG,131
4,2023-12-10,REGIONSHOSPITALET GØDSTRUP,KVINDESYGDOMME OG FØDSLER - RHG,33


#### Add number of patients contacts at the department

In [5]:
# Convert date to timestamp and remove the date column
patient_contact["Timestamp"] = pd.to_datetime(patient_contact["Dato_Ind"])
patient_contact = patient_contact.drop(["Dato_Ind","Nuv_OrgMapSghTekst"], axis = 1)

In [6]:
patient_contact.head()

,Nuv_OrgMapOverAfdTekst,Antal_Kontakter,Timestamp
0,MEDICINSK AFDELING - RHG,309,2022-03-24
1,URINVEJSKIRURGI - RHG,57,2021-07-19
2,URINVEJSKIRURGI - RHG,85,2020-08-06
3,URINVEJSKIRURGI - RHG,131,2019-12-04
4,KVINDESYGDOMME OG FØDSLER - RHG,33,2023-12-10


In [7]:
# Remove "- RHG" from the column "Nuv_OrgMapOverAfdTekst" and make to lowercase
patient_contact["Nuv_OrgMapOverAfdTekst"] = patient_contact["Nuv_OrgMapOverAfdTekst"].str.replace(r' - RHG', '', regex=True).str.lower()

In [8]:
# Rename column
patient_contact = patient_contact.rename(columns={"Nuv_OrgMapOverAfdTekst": "Afdelinger"})
data = data.rename(columns={"LoenafdelingNiveau2Tekst":"Afdelinger"})

In [11]:
# Convert values in department columns
def replace_department(idx):
    afd = data["Afdelinger"][idx].lower()
    #print(afd)
    for j in patient_contact["Afdelinger"].unique():
        if afd.startswith(j)==True:
            #print(j)
            val = j
            break
        else: val = afd
    return(val)

In [ ]:
# Convert strings to match pattern
for i in range(len(data["Afdelinger"])):   
    j = replace_department(i)
    data.loc[i,"Afdelinger"] = j

In [19]:
# Convert to datetime object
data["Timestamp"] = pd.to_datetime(data["Timestamp"])

In [20]:
# Merge dfs
merged_df = pd.merge(data, patient_contact, on=['Afdelinger', 'Timestamp'], how = "left")
merged_df.head()

In [24]:
# Check NAs and replace with 0
print(merged_df["Antal_Kontakter"].isna().sum())

merged_df["Antal_Kontakter"] = merged_df["Antal_Kontakter"].fillna(0)


42050


In [26]:
# Write csv
merged_df.to_csv("/work/Exam/data_w_patients.csv")